In [ ]:
"""
SIFT_Classifier_R3.0.py: EEG Classifier for Epilepsy and Parkinson's Research
- Classifies EEG features as normal, seizure, or Parkinson’s-like using logistic regression.
- Inputs: Feature arrays from SIFT_Analysis_R3.0.py or other sources.
- Outputs: Classification labels, confusion matrix, logs.
- Dependencies: numpy, sklearn, seaborn, matplotlib.
- License: GNU GPL v3.0
- Ethical Statement: For research only, no therapeutic use.
"""

import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Setup directories and logging
input_dir = './output_sift_analysis/'
output_dir = './output_sift_classifier/'
os.makedirs(output_dir, exist_ok=True)
logging.basicConfig(filename=os.path.join(output_dir, 'sift_classifier.log'), level=logging.INFO)

# Load features (example: from SIFT_Analysis_R3.0.py)
try:
    features_e = np.load(os.path.join(input_dir, 'features_epilepsy.npy'))
    features_n = np.load(os.path.join(input_dir, 'features_normal.npy'))
    features_p = np.load(os.path.join(input_dir, 'features_parkinsons.npy'))
except FileNotFoundError as e:
    raise FileNotFoundError(f"Could not find {e.filename}. Ensure features are in {input_dir}.")

# Combine features and labels
features = np.vstack([features_e[np.newaxis, :], features_n[np.newaxis, :], features_p[np.newaxis, :]])
labels = ['seizure', 'normal', 'parkinsons']

# Classifier
scaler = StandardScaler().fit(features)
model = LogisticRegression(class_weight={'normal': 1, 'seizure': 3, 'parkinsons': 3})
model.fit(scaler.transform(features), labels)
predictions = model.predict(scaler.transform(features))

# Evaluate
cm = confusion_matrix(labels, predictions, labels=['normal', 'seizure', 'parkinsons'])
accuracy = np.trace(cm) / np.sum(cm)
logging.info(f"Classifier Accuracy: {accuracy:.2%}")
logging.info(f"Confusion Matrix:\n{cm}")
print(f"Classifier Accuracy: {accuracy:.2%}")
print(f"Confusion Matrix:\n{cm}")

# Plotting
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Normal', 'Seizure', 'Parkinson’s'], yticklabels=['Normal', 'Seizure', 'Parkinson’s'])
plt.title('Confusion Matrix for SIFT Classifier')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
plt.close()

# Save results
np.save(os.path.join(output_dir, 'predictions.npy'), predictions)

print(f"SIFT classification complete. Results saved to {output_dir}")